In [4]:
import requests
import random 
import pandas as pd
from bs4 import BeautifulSoup
import time
import json

#import ast
from credentials import API_KEY


In [14]:
#Leemos los datos tomados datos abiertos colombia, actualizados por ultima vez 2022-06
df_2 = pd.read_excel('../data_files//dataset_precios_combustibles.xlsx')
df_2

,Unnamed: 0,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado
0,0,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA EXTRA OXIGENADA,17880.0,A
1,1,2020,12,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,BIODIESEL EXTRA,8190.0,A
2,2,2020,12,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,BIODIESEL EXTRA,8190.0,A
3,3,2020,12,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA EXTRA OXIGENADA,10790.0,A
4,4,2020,12,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA CORRIENTE OXIGENADA,8120.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
584794,584794,2022,5,99,VICHADA,99773,CUMARIBO,ESTACION DE SERVICIO AIPORE,TERPEL,Carrera 10 Calle 7 Esquina,GASOLINA CORRIENTE OXIGENADA,10600.0,A
584795,584795,2022,5,99,VICHADA,99773,CUMARIBO,ESTACION DE SERVICIO AIPORE,TERPEL,Carrera 10 Calle 7 Esquina,BIODIESEL EXTRA,9300.0,A
584796,584796,2022,5,99,VICHADA,99773,CUMARIBO,ESTACION DE SERVICIO AIPORE,TERPEL,Carrera 10 Calle 7 Esquina,GASOLINA CORRIENTE OXIGENADA,9100.0,A
584797,584797,2022,5,99,VICHADA,99773,CUMARIBO,ESTACION DE SERVICIO AIPORE,TERPEL,Carrera 10 Calle 7 Esquina,GASOLINA CORRIENTE OXIGENADA,9990.0,A


In [3]:

df = df_2.drop(['Unnamed: 0'], axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584799 entries, 0 to 584798
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Periodo              584799 non-null  int64  
 1   Mes                  584799 non-null  int64  
 2   Codigo_departamento  584799 non-null  int64  
 3   Departamento         584799 non-null  object 
 4   Codigo_municipio     584799 non-null  int64  
 5   Municipio            584799 non-null  object 
 6   Nombre_comercial     584799 non-null  object 
 7   Bandera              584799 non-null  object 
 8   Direccion            584799 non-null  object 
 9   Producto             584799 non-null  object 
 10  Precio               584799 non-null  float64
 11  Estado               584799 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 53.5+ MB


In [5]:
df.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado
0,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA EXTRA OXIGENADA,17880.0,A
1,2020,12,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,BIODIESEL EXTRA,8190.0,A


In [6]:
df[df['Periodo'] == 2022]['Mes'].unique()

array([6, 4, 2, 1, 3, 5])

In [7]:
#Filtramos para obtener los precios del ultimo periodo
df_3 = df[(df['Periodo'] == 2022)&(df['Mes'] == 6)]
df_3.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado
0,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA EXTRA OXIGENADA,17880.0,A
391910,2022,6,5,ANTIOQUIA,5607,RETIRO,LA FE,TEXACO,centro comercial la fe,BIODIESEL EXTRA,9780.0,A


In [8]:
df_3['Departamento'].value_counts()

Departamento
NARIÃ‘O                                                     3968
ANTIOQUIA                                                   2720
VALLE DEL CAUCA                                             2294
BOGOTA D.C.                                                 1818
CUNDINAMARCA                                                1789
NORTE DE SANTANDER                                          1573
CESAR                                                       1245
SANTANDER                                                   1168
TOLIMA                                                      1060
BOYACA                                                       887
HUILA                                                        757
ATLANTICO                                                    717
PUTUMAYO                                                     700
LA GUAJIRA                                                   683
RISARALDA                                                    679
META        

In [9]:
antioquia_df = df_3[(df_3['Departamento'] == 'ANTIOQUIA')]

In [10]:
municipios = list(antioquia_df['Municipio'].value_counts().head(6).reset_index().Municipio.unique())

In [11]:
municipios

['MEDELLIN', 'ITAGUI', 'BELLO', 'SABANETA', 'RIONEGRO', 'ENVIGADO']

In [12]:
#Filtramos para obtener solo las gasolineras Medellin, Itagui, Bello, Sabaneta, Rionegro y Envigado
antioquia_filtered_df= antioquia_df[(antioquia_df['Municipio'] == 'MEDELLIN') | 
             (antioquia_df['Municipio'] == 'ITAGUI') |
             (antioquia_df['Municipio'] == 'BELLO') |
             (antioquia_df['Municipio'] == 'SABANETA') |
             (antioquia_df['Municipio'] == 'RIONEGRO') |
             (antioquia_df['Municipio'] == 'ENVIGADO') ]

In [13]:
antioquia_filtered_df.reset_index(inplace=True)
antioquia_filtered_df

,index,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado
0,0,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA EXTRA OXIGENADA,17880.0,A
1,536459,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,BIODIESEL EXTRA,9040.0,A
2,536605,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA CORRIENTE OXIGENADA,9010.0,A
3,536606,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,BIODIESEL EXTRA,9040.0,A
4,536607,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA CORRIENTE OXIGENADA,8990.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,538925,2022,6,5,ANTIOQUIA,5631,SABANETA,EDS COMDECOM SANTA MARIA,TEXACO,CRA 48 AV LAS VEGAS ENTRE CL 72 SUR Y CL 73 SUR,BIODIESEL EXTRA,9570.0,A
1292,538926,2022,6,5,ANTIOQUIA,5631,SABANETA,EDS COMDECOM SANTA MARIA,TEXACO,CRA 48 AV LAS VEGAS ENTRE CL 72 SUR Y CL 73 SUR,BIODIESEL EXTRA,9570.0,A
1293,538927,2022,6,5,ANTIOQUIA,5631,SABANETA,EDS COMDECOM SANTA MARIA,TEXACO,CRA 48 AV LAS VEGAS ENTRE CL 72 SUR Y CL 73 SUR,BIODIESEL EXTRA,9540.0,A
1294,538928,2022,6,5,ANTIOQUIA,5631,SABANETA,EDS COMDECOM SANTA MARIA,TEXACO,CRA 48 AV LAS VEGAS ENTRE CL 72 SUR Y CL 73 SUR,GASOLINA CORRIENTE OXIGENADA,9780.0,A


In [14]:
antioquia_filtered_df.drop(columns='index', inplace=True)
antioquia_filtered_df

/tmp/ipykernel_374/1225709700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antioquia_filtered_df.drop(columns='index', inplace=True)


,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado
0,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA EXTRA OXIGENADA,17880.0,A
1,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,BIODIESEL EXTRA,9040.0,A
2,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA CORRIENTE OXIGENADA,9010.0,A
3,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,BIODIESEL EXTRA,9040.0,A
4,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA CORRIENTE OXIGENADA,8990.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...
1291,2022,6,5,ANTIOQUIA,5631,SABANETA,EDS COMDECOM SANTA MARIA,TEXACO,CRA 48 AV LAS VEGAS ENTRE CL 72 SUR Y CL 73 SUR,BIODIESEL EXTRA,9570.0,A
1292,2022,6,5,ANTIOQUIA,5631,SABANETA,EDS COMDECOM SANTA MARIA,TEXACO,CRA 48 AV LAS VEGAS ENTRE CL 72 SUR Y CL 73 SUR,BIODIESEL EXTRA,9570.0,A
1293,2022,6,5,ANTIOQUIA,5631,SABANETA,EDS COMDECOM SANTA MARIA,TEXACO,CRA 48 AV LAS VEGAS ENTRE CL 72 SUR Y CL 73 SUR,BIODIESEL EXTRA,9540.0,A
1294,2022,6,5,ANTIOQUIA,5631,SABANETA,EDS COMDECOM SANTA MARIA,TEXACO,CRA 48 AV LAS VEGAS ENTRE CL 72 SUR Y CL 73 SUR,GASOLINA CORRIENTE OXIGENADA,9780.0,A


In [15]:
antioquia_filtered_df.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado
0,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA EXTRA OXIGENADA,17880.0,A
1,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,BIODIESEL EXTRA,9040.0,A


In [16]:
#Modificamos los str de la columna direccion para que sea compatible con el formato que especifica la API
antioquia_filtered_df['Direccion_API'] = antioquia_filtered_df['Direccion'].apply(lambda x: x.replace('#','No'))
antioquia_filtered_df['Direccion_API'] = antioquia_filtered_df.apply(lambda x : x['Direccion_API']+', '+x['Municipio'].capitalize()+', Colombia', axis=1)

/tmp/ipykernel_374/2462176317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antioquia_filtered_df['Direccion_API'] = antioquia_filtered_df['Direccion'].apply(lambda x: x.replace('#','No'))
/tmp/ipykernel_374/2462176317.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antioquia_filtered_df['Direccion_API'] = antioquia_filtered_df.apply(lambda x : x['Direccion_API']+', '+x['Municipio'].capitalize()+', Colombia', axis=1)


In [17]:
antioquia_filtered_df.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Direccion_API
0,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA EXTRA OXIGENADA,17880.0,A,"CARRERA 45 No. 66-10, Medellin, Colombia"
1,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,BIODIESEL EXTRA,9040.0,A,"CARRERA 45 No. 66-10, Medellin, Colombia"


In [18]:
#creamos un df solo con las gasolineras que vamos a tener en cuenta para el analisis
df_final = antioquia_filtered_df[['Nombre_comercial','Direccion','Direccion_API']].copy(deep=True)
df_final.drop_duplicates(subset='Nombre_comercial', inplace=True)
df_final

,Nombre_comercial,Direccion,Direccion_API
0,ESTACION DE SERVICIO TERPEL LA 45,CARRERA 45 No. 66-10,"CARRERA 45 No. 66-10, Medellin, Colombia"
6,ESTACION DE SERVICIO LA AGUACATALA,Carrera 48 No 15 sur 30,"Carrera 48 No 15 sur 30, Medellin, Colombia"
12,ESTACION DE SERVICIO JARDIN,Carrera 51 No.73-20 San Pedro,"Carrera 51 No.73-20 San Pedro, Medellin, Colombia"
14,AUTOCENTRO LA PALMA 2,CALLE 28 No 80A-32,"CALLE 28 No 80A-32, Medellin, Colombia"
17,APARCADERO TASSA,Carrera 51B No 87 - 29,"Carrera 51B No 87 - 29, Medellin, Colombia"
...,...,...,...
1236,PAYSANDU E-S MAYORCA,CALLE 57 SUR No. 48-25,"CALLE 57 SUR No. 48-25, Sabaneta, Colombia"
1251,ESTACION DE SERVICIO MARCELLA,CALLE 84 SUR No. 33-161,"CALLE 84 SUR No. 33-161, Sabaneta, Colombia"
1263,ESTACION DE SERVICIO DISTRACOM MARIA AUXILIADORA,CARRERA 45 No 61 SUR - 53,"CARRERA 45 No 61 SUR - 53, Sabaneta, Colombia"
1275,ESTACION DE SERVICIO EXITO VARIANTE CALDAS,CALLE 84 SUR No. 29-175,"CALLE 84 SUR No. 29-175, Sabaneta, Colombia"


In [19]:
df_final.reset_index(inplace=True)
df_final.drop(columns='index', inplace=True)


In [20]:
df_final

,Nombre_comercial,Direccion,Direccion_API
0,ESTACION DE SERVICIO TERPEL LA 45,CARRERA 45 No. 66-10,"CARRERA 45 No. 66-10, Medellin, Colombia"
1,ESTACION DE SERVICIO LA AGUACATALA,Carrera 48 No 15 sur 30,"Carrera 48 No 15 sur 30, Medellin, Colombia"
2,ESTACION DE SERVICIO JARDIN,Carrera 51 No.73-20 San Pedro,"Carrera 51 No.73-20 San Pedro, Medellin, Colombia"
3,AUTOCENTRO LA PALMA 2,CALLE 28 No 80A-32,"CALLE 28 No 80A-32, Medellin, Colombia"
4,APARCADERO TASSA,Carrera 51B No 87 - 29,"Carrera 51B No 87 - 29, Medellin, Colombia"
...,...,...,...
173,PAYSANDU E-S MAYORCA,CALLE 57 SUR No. 48-25,"CALLE 57 SUR No. 48-25, Sabaneta, Colombia"
174,ESTACION DE SERVICIO MARCELLA,CALLE 84 SUR No. 33-161,"CALLE 84 SUR No. 33-161, Sabaneta, Colombia"
175,ESTACION DE SERVICIO DISTRACOM MARIA AUXILIADORA,CARRERA 45 No 61 SUR - 53,"CARRERA 45 No 61 SUR - 53, Sabaneta, Colombia"
176,ESTACION DE SERVICIO EXITO VARIANTE CALDAS,CALLE 84 SUR No. 29-175,"CALLE 84 SUR No. 29-175, Sabaneta, Colombia"


In [21]:
dir =df_final['Direccion_API'][0]

In [22]:
url_geo = f'https://geocode.search.hereapi.com/v1/geocode?q={dir}&apiKey={API_KEY}'

NameError: name 'api_key_geocoding' is not defined

In [ ]:
response = requests.get(url=url_geo).json()
response

{'items': [{'title': 'Carrera 45A-10, 050003 Medellín, Colombia',
   'id': 'here:af:streetsection:i28LPR2oS6pY9dcD3AMbCB:CggIBCC69uv4AhABGgIxMCIFOTUtMTA',
   'resultType': 'houseNumber',
   'houseNumberType': 'PA',
   'address': {'label': 'Carrera 45A-10, 050003 Medellín, Colombia',
    'countryCode': 'COL',
    'countryName': 'Colombia',
    'countyCode': 'AN',
    'county': 'Antioquia',
    'city': 'Medellín',
    'district': 'Popular',
    'street': 'Carrera 45A',
    'postalCode': '050003',
    'houseNumber': '10',
    'building': '95-10'},
   'position': {'lat': 6.28685, 'lng': -75.55267},
   'access': [{'lat': 6.28685, 'lng': -75.55267}],
   'mapView': {'west': -75.55357,
    'south': 6.28595,
    'east': -75.55177,
    'north': 6.28775},
   'scoring': {'queryScore': 0.87,
    'fieldScore': {'country': 1.0,
     'city': 1.0,
     'streets': [1.0],
     'houseNumber': 1.0}}}]}

In [ ]:
response['items'][0]['position']['lng']

-75.55267

In [ ]:
def get_coords(address,api_key_geocoding):
    
    url_get = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key_geocoding}'
    
    try:
        response = requests.get(url=url_get).json()
        clean_address = response['items'][0]['title']
        coord_lat = response['items'][0]['position']['lat']
        coord_lng = response['items'][0]['position']['lng']
        
        results = [clean_address, coord_lat, coord_lng]
    except:
        
        results = ['Not Found', 'NA', 'NA']
    
    return results
    


In [ ]:
get_coords(df_final['Direccion_API'][1], API_KEY)

['Carrera 48 15 S-30, 050022 Medellín, Colombia', 6.19221, -75.58]

In [ ]:
coordenadas = []
tiempo = [1,2,3,4]

for i in range(len(df_final.index)):
    coor = get_coords(df_final['Direccion_API'][i], API_KEY)
    coor.append(df_final['Direccion_API'][i])
    coordenadas.append(coor)
    time.sleep(random.choice(tiempo))
    
coordenadas

[['Carrera 45A-10, 050003 Medellín, Colombia',
  6.28685,
  -75.55267,
  'CARRERA 45 No. 66-10, Medellin, Colombia'],
 ['Carrera 48 15 S-30, 050022 Medellín, Colombia',
  6.19221,
  -75.58,
  'Carrera 48 No 15 sur 30, Medellin, Colombia'],
 ['Carrera 51-20, 050010 Medellín, Colombia',
  6.26982,
  -75.56186,
  'Carrera 51 No.73-20 San Pedro, Medellin, Colombia'],
 ['Calle 28 80A-32, 050025 Medellín, Colombia',
  6.22945,
  -75.60168,
  'CALLE 28 No 80A-32, Medellin, Colombia'],
 ['Carrera 51B 87-29, 050004 Medellín, Colombia',
  6.28006,
  -75.56187,
  'Carrera 51B  No 87 - 29, Medellin, Colombia'],
 ['Calle 45A 60-13, 050015 Medellín, Colombia',
  6.24928,
  -75.57796,
  'Calle 45A No 60 - 13, Medellin, Colombia'],
 ['Avenida 33-145, 050030 Medellín, Colombia',
  6.2388,
  -75.59817,
  'AVENIDA 33 No. 75C-145, Medellin, Colombia'],
 ['Carrera 50 40-77, 050015 Medellín, Colombia',
  6.24137,
  -75.5712,
  'CARRERA 50 No. 40-77, Medellin, Colombia'],
 ['Calle 49A-25, 050013 Medellín, Co

In [ ]:
#obtenemos el df con las coordenadas
coord_df = pd.DataFrame(coordenadas, columns=['Direcion','Latitud', 'Longitud','Direccion_API'])
coord_df

,Direcion,Latitud,Longitud,Direccion_API
0,"Carrera 45A-10, 050003 Medellín, Colombia",6.28685,-75.55267,"CARRERA 45 No. 66-10, Medellin, Colombia"
1,"Carrera 48 15 S-30, 050022 Medellín, Colombia",6.19221,-75.58,"Carrera 48 No 15 sur 30, Medellin, Colombia"
2,"Carrera 51-20, 050010 Medellín, Colombia",6.26982,-75.56186,"Carrera 51 No.73-20 San Pedro, Medellin, Colombia"
3,"Calle 28 80A-32, 050025 Medellín, Colombia",6.22945,-75.60168,"CALLE 28 No 80A-32, Medellin, Colombia"
4,"Carrera 51B 87-29, 050004 Medellín, Colombia",6.28006,-75.56187,"Carrera 51B No 87 - 29, Medellin, Colombia"
...,...,...,...,...
173,"Calle 57 Sur 48-25, 055450 Sabaneta, Colombia",6.15851,-75.60893,"CALLE 57 SUR No. 48-25, Sabaneta, Colombia"
174,"Calle 75 Sur 33-161, 055450 Sabaneta, Colombia",6.14141,-75.61305,"CALLE 84 SUR No. 33-161, Sabaneta, Colombia"
175,"Carrera 43A 61 S-53, 055450 Sabaneta, Colombia",6.15429,-75.60949,"CARRERA 45 No 61 SUR - 53, Sabaneta, Colombia"
176,"Calle 65 Sur 38-175, 055450 Sabaneta, Colombia",6.14839,-75.61016,"CALLE 84 SUR No. 29-175, Sabaneta, Colombia"


In [ ]:
#hacemos un merge entre el df original y el df con los coordenadas de cada gasolinera
df_cor_final = pd.merge(
    antioquia_filtered_df,
    coord_df[['Latitud', 'Longitud','Direccion_API']],
    how='inner',
    left_on='Direccion_API',
    right_on='Direccion_API')

In [ ]:
df_cor_final['Latitud'].isna().value_counts()


Latitud
False    1293
Name: count, dtype: int64

In [ ]:
df_cor_final.dropna(inplace=True)


In [ ]:
#exportamos el archivo
df_cor_final.to_csv('data_set_OK.csv', index=False)